In [1]:
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
from Invtsim import *
import numpy as np
import pandas as pd
from scipy.stats import norm
import warnings
warnings.simplefilter("ignore")
from tqdm import tqdm

gap1 = 365
gap2 = 1548

train    = pd.read_pickle(f"{path1}721past_1913.pkl").reset_index(drop=True)[['ds','y']]
fit_all  = pd.read_pickle(f"{path1}721fitts_base2cohe.pkl")
tr_all   = pd.concat([train,fit_all], axis =1)

tr_tr    = tr_all[tr_all['ds'] <= gap2].reset_index(drop=True).iloc[:,1:]
tr_ts    = tr_all[tr_all['ds'] >  gap2].reset_index(drop=True).iloc[:,1:]

## Hist Order:

In [2]:
def loop(fcst, truth, residual, NAME:str,
         gap1: int = 28, gap2:int = 1913,
         n = None, 
         df_list:list = [], 
         L_ = 3):
    if n == None:
        n = int(len(fcst)/gap1)
    for i in tqdm(range(n)):
        fcst_1  = fcst[i*gap1:(i+1)*gap1]
        truth_1 = truth[i*gap1:(i+1)*gap1]
        res_1   = residual[i*gap2:(i+1)*gap2]
        df = InvtSim(fcst = fcst_1, truth = truth_1, residual = res_1, name = NAME, L=L_, period = gap1)
        res = df.ob_all_t()
        df_list.append(res.reset_index(drop = True))
    #results = pd.concat(res)
    print("-----------DOWN!------------")
    return pd.concat(df_list).reset_index(drop=True)

In [ ]:
#lead_time = 1
fcst365, actl365 = {},{}
for lead_time in [1,3, 7,14]:
    print(f"************Lead time is: {lead_time}************")
    gap1, gap2 = 365,1548
    col = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu','ets_td', 'ets_mint']
    trtsy = tr_ts['y']
    trtry = tr_tr['y']
    trInvtSim = []
    for i in col:
        df = loop(fcst = trtsy, truth = trtsy, residual = trtry-trtry, NAME = f'tr_{i}', df_list = [],L_=lead_time, gap1 = gap1, gap2 = gap2)
        trInvtSim.append(df[['name','ot_90','ot_95','ot_99']])
    actl365_L = pd.concat([trInvtSim[i] for i in range(8)])
    actl365_L[['ot_90','ot_95','ot_99']] = actl365_L[['ot_90','ot_95','ot_99']].clip(lower=0)
    actl365_L.to_pickle(f"721_365TRactual_L{lead_time}.pkl")
    #pd.read_pickle("365TRactual.pkl")
    trtsy = tr_ts['y']
    trtry = tr_tr['y']
    trInvtSim = []
    for i in col:
        df = loop(fcst = tr_ts[i], truth = trtsy, residual = tr_tr[i]-trtry, NAME = f'tr_{i}', df_list = [],
              L_=lead_time, gap1 = gap1, gap2 = gap2)
        trInvtSim.append(df[['name','ot_90','ot_95','ot_99']])
    fcst365_L = pd.concat([trInvtSim[i] for i in range(8)])
    fcst365_L[['ot_90','ot_95','ot_99']] = fcst365_L[['ot_90','ot_95','ot_99']].clip(lower=0)
    fcst365_L.to_pickle(f"721_365TRfcst_L{lead_time}.pkl")
    
#actl365_L.to_pickle(f'365TRactual_L{lead_time}.pkl')
#fcst365_L.to_pickle(f'365TRfcst_L{lead_time}.pkl')

In [3]:
import pandas as pd
lead_time = 1
actl365_L = pd.read_pickle(f'721_365TRactual_L{lead_time}.pkl')
fcst365_L = pd.read_pickle(f'721_365TRfcst_L{lead_time}.pkl')

In [10]:
fcst365_L['name'].unique()

array(['tr_lgb_base', 'tr_lgb_bu', 'tr_lgb_td', 'tr_lgb_mint',
       'tr_ets_base', 'tr_ets_bu', 'tr_ets_td', 'tr_ets_mint'],
      dtype=object)

In [9]:
365*42686

15580390

## Inventory Reconciliation

In [3]:
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
import warnings
warnings.simplefilter("ignore")
from Invtsim import *
import numpy as np
import pandas as pd
from scipy.stats import norm
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace
from tqdm import tqdm

S       = pd.read_pickle(f"{path1}721S_df.pkl")
tags    = pd.read_pickle(f"{path1}tags.bin")

'''lp = {"l1" :(0,1),
      "l2" :(1,(1+3)),
      "l4" :((1+3),(1+3+3)),
      "l3" :((1+3+3),(1+3+3+10)),
      "l5" :((1+3+10+3),(1+3+10+3+7)),
      "l6" :((1+3+10+3+7),(1+3+10+3+7+9)),
      "l7" :((1+3+10+3+7+9),(1+3+10+3+7+9+21)),
      "l8" :((1+3+10+3+7+9+21),(1+3+10+3+7+9+21+30)),
      "l10" :((1+3+10+3+7+9+21+30),(1+3+10+3+7+9+21+30+3049)),
      "l9":((1+3+10+3+7+9+21+30+3049),(1+3+10+3+7+9+21+30+3049+70)),
      "l11":((1+3+10+3+7+9+21+30+70+3049),(1+3+10+3+7+9+21+30+70+3049+9147)),
      "l12":((1+3+10+3+7+9+21+30+70+3049+9147),(1+3+10+3+7+9+21+30+70+3049+9147+30490))}'''
lp = {"l10" :(0,3049),"l11":(3049,3049+9147),"l12":(3049+9147,3049+9147+30490)}


s = pd.concat([S.iloc[lp[i][0]:lp[i][1],:] for i in ["l10","l11","l12"]]) 
taglist = ['total/cat_id/dept_id/item_id',
           'total/state_id/cat_id/dept_id/item_id',
           'total/state_id/store_id/cat_id/dept_id/item_id']
tags_ = {i: tags[i] for i in taglist}

gap1    = 365
gap2    = 1548
train   = pd.read_pickle(f"{path1}721past_1913.pkl").reset_index(drop=True)
test    = pd.read_pickle(f"{path1}721future_28.pkl").reset_index(drop=True)


ts_id   = test[['unique_id', 'ds']]
tr_id   = train[['unique_id', 'ds']]
trts_id = tr_id[tr_id['ds']> gap2].reset_index(drop=True)

ts_id  = pd.concat([ts_id.iloc[lp[i][0]*28:lp[i][1]*28,:] for i in ["l10","l11","l12"]])
tr_id  = pd.concat([trts_id.iloc[lp[i][0]*365:lp[i][1]*365,:] for i in ["l10","l11","l12"]])

#trtr_id = tr_id[tr_id['ds']<= gap2]

In [8]:
lead_time = 1
tr_fo   = pd.read_pickle(f"721_365TRfcst_L{lead_time}.pkl") ########################################################jiushuo
tr_do   = pd.read_pickle(f"721_365TRactual_L{lead_time}.pkl").iloc[:,:2]################################################
tr_do.columns = ['name','y']
tr365   = pd.concat([tr_do, tr_fo[['ot_90','ot_95','ot_99']]],axis=1)
tr365   = pd.concat([tr365[tr365['name'] == i].iloc[tr_id.index,:] for i in  ['tr_lgb_base', 'tr_lgb_mint', 'tr_ets_base',  'tr_ets_mint']])


In [10]:
tr_fo['name'].unique()

array(['tr_lgb_base', 'tr_lgb_bu', 'tr_lgb_td', 'tr_lgb_mint',
       'tr_ets_base', 'tr_ets_bu', 'tr_ets_td', 'tr_ets_mint'],
      dtype=object)

## Order MinT

In [12]:
%%time
lead_time = 1
method = "ols"
lgbs    =  pd.read_pickle(f"721lgbInvtSim_L{lead_time}.pkl")
etss    = pd.read_pickle(f"721etsInvtSim_L{lead_time}.pkl")
MMint   = pd.concat([lgbs,etss])[['name','ot_90','ot_95','ot_99']]
MMint   = pd.concat([MMint[MMint['name'] == i].iloc[ts_id.index,:] for i in  ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 
                                                                              'ets_base', 'ets_bu', 'ets_td', 'ets_mint']])

on   = ['ot_90','ot_95','ot_99']
nn = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu', 'ets_td', 'ets_mint']
#nn = ['lgb_base', 'lgb_mint', 'ets_base',  'ets_mint']
minimumtrace = [MinTrace(method = 'ols')]
mint = HierarchicalReconciliation(reconcilers=minimumtrace)
MINTo = []
for i in tqdm(range(len(nn))):
    lgb_omt = pd.concat([ts_id, MMint[MMint['name'] == f'{nn[i]}'][on]],axis=1)
    tr_order= pd.concat([tr_id.reset_index(drop=True), 
                         tr365[tr365['name'] == f"tr_{nn[i]}"].reset_index(drop=True)],axis=1).drop(columns = ['name'])
    subset = []
    for j in on:
        subset.append(mint.reconcile(Y_hat_df=lgb_omt[['unique_id','ds',j]], S=s, tags=tags_,  Y_df= tr_order[['unique_id','ds','y',j]]).iloc[:,-1:])
    MINTo.append(pd.concat(subset, axis=1))

MinT = pd.concat([MMint.loc[MMint.iloc[:,0].isin(nn),'name'].reset_index(drop=True), 
                  pd.concat(MINTo).reset_index(drop=True)], axis = 1)
#col = MinT.columns[-3:]
#MinT[col] =MinT[col].clip(lower=0)
MinT.to_pickle(f"721OrderMinT_{method}_L{lead_time}.pkl")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [1:42:55<00:00, 771.96s/it]


Wall time: 1h 43min 5s


### L3

In [5]:
%%time
lead_time = 3
method = "ols"
lgbs    =  pd.read_pickle(f"lgbInvtSim_L{lead_time}.pkl")
etss    = pd.read_pickle(f"etsInvtSim_L{lead_time}.pkl")
MMint   = pd.concat([lgbs,etss])[['name','ot_90','ot_95','ot_99']]
MMint   = pd.concat([MMint[MMint['name'] == i].iloc[ts_id.index,:] for i in  ['lgb_base', 'lgb_mint', 'ets_base',  'ets_mint']])

tr_fo   = pd.read_pickle(f"365TRfcst_L{lead_time}.pkl") ########################################################jiushuo
tr_do   = pd.read_pickle(f"365TRactual_L{lead_time}.pkl").iloc[:,:2]################################################
tr_do.columns = ['name','y']
tr365   = pd.concat([tr_do, tr_fo[['ot_90','ot_95','ot_99']]],axis=1)
tr365   = pd.concat([tr365[tr365['name'] == i].iloc[tr_id.index,:] for i in  ['tr_lgb_base', 'tr_lgb_mint', 'tr_ets_base',  'tr_ets_mint']])

on   = ['ot_90','ot_95','ot_99']
#nn = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu', 'ets_td', 'ets_mint']
nn = ['lgb_base', 'lgb_mint', 'ets_base',  'ets_mint']
minimumtrace = [MinTrace(method = 'ols')]
mint = HierarchicalReconciliation(reconcilers=minimumtrace)
MINTo = []
for i in tqdm(range(len(nn))):
    lgb_omt = pd.concat([ts_id, MMint[MMint['name'] == f'{nn[i]}'][on]],axis=1)
    tr_order= pd.concat([tr_id.reset_index(drop=True), 
                         tr365[tr365['name'] == f"tr_{nn[i]}"].reset_index(drop=True)],axis=1).drop(columns = ['name'])
    subset = []
    for j in on:
        subset.append(mint.reconcile(Y_hat_df=lgb_omt[['unique_id','ds',j]], 
                                     S=s, tags=tags_,  Y_df= tr_order[['unique_id','ds','y',j]]).iloc[:,-1:])
    MINTo.append(pd.concat(subset, axis=1))

MinT = pd.concat([MMint.loc[MMint.iloc[:,0].isin(nn),'name'].reset_index(drop=True), 
                  pd.concat(MINTo).reset_index(drop=True)], axis = 1)
#col = MinT.columns[-3:]
#MinT[col] =MinT[col].clip(lower=0)
MinT.to_pickle(f"OrderMinT_L{lead_time}_{method}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [49:01<00:00, 735.32s/it]


Wall time: 50min 7s


### L7

In [6]:
%%time
lead_time = 7
method = "ols"
lgbs    =  pd.read_pickle(f"lgbInvtSim_L{lead_time}.pkl")
etss    = pd.read_pickle(f"etsInvtSim_L{lead_time}.pkl")
MMint   = pd.concat([lgbs,etss])[['name','ot_90','ot_95','ot_99']]
MMint   = pd.concat([MMint[MMint['name'] == i].iloc[ts_id.index,:] for i in  ['lgb_base', 'lgb_mint', 'ets_base',  'ets_mint']])

tr_fo   = pd.read_pickle(f"365TRfcst_L{lead_time}.pkl") ########################################################jiushuo
tr_do   = pd.read_pickle(f"365TRactual_L{lead_time}.pkl").iloc[:,:2]################################################
tr_do.columns = ['name','y']
tr365   = pd.concat([tr_do, tr_fo[['ot_90','ot_95','ot_99']]],axis=1)
tr365   = pd.concat([tr365[tr365['name'] == i].iloc[tr_id.index,:] for i in  ['tr_lgb_base', 'tr_lgb_mint', 'tr_ets_base',  'tr_ets_mint']])

on   = ['ot_90','ot_95','ot_99']
#nn = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu', 'ets_td', 'ets_mint']
nn = ['lgb_base', 'lgb_mint', 'ets_base',  'ets_mint']
minimumtrace = [MinTrace(method = 'ols')]
mint = HierarchicalReconciliation(reconcilers=minimumtrace)
MINTo = []
for i in tqdm(range(len(nn))):
    lgb_omt = pd.concat([ts_id, MMint[MMint['name'] == f'{nn[i]}'][on]],axis=1)
    tr_order= pd.concat([tr_id.reset_index(drop=True), 
                         tr365[tr365['name'] == f"tr_{nn[i]}"].reset_index(drop=True)],axis=1).drop(columns = ['name'])
    subset = []
    for j in on:
        subset.append(mint.reconcile(Y_hat_df=lgb_omt[['unique_id','ds',j]], S=s, tags=tags_,  
                                     Y_df= tr_order[['unique_id','ds','y',j]]).iloc[:,-1:])
    MINTo.append(pd.concat(subset, axis=1))

MinT = pd.concat([MMint.loc[MMint.iloc[:,0].isin(nn),'name'].reset_index(drop=True), 
                  pd.concat(MINTo).reset_index(drop=True)], axis = 1)
#col = MinT.columns[-3:]
#MinT[col] =MinT[col].clip(lower=0)
MinT.to_pickle(f"OrderMinT_L{lead_time}_{method}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [48:52<00:00, 733.06s/it]


Wall time: 49min 57s


## Fixed Order Simulation:

In [3]:
pip install "numpy<2" --upgrade --user -i https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple numpy

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


In [6]:
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
from Invtsim import *
import numpy as np
import pandas as pd
from scipy.stats import norm
import warnings
warnings.simplefilter("ignore")
from tqdm import tqdm
def loop2( truth, NAME:str, fcst, residual = 0,
         gap1: int = 28, gap2:int = 1913,
         n = None, 
         df_list:list = [], 
         L_ = 3):
    if n == None:
        n = int(len(truth)/gap1)
    for i in tqdm(range(n)):
        fcst_1  = fcst[i*gap1:(i+1)*gap1]
        truth_1 = truth[i*gap1:(i+1)*gap1]
        #res_1   = residual[i*gap2:(i+1)*gap2]
        df = InvtSim(fcst = 0, truth = truth_1, residual = residual, name = NAME, L=L_)
        res = df.ob_all_t_fixedcase(fixed_order = fcst_1)
        df_list.append(res.reset_index(drop = True))
    print("-----------DOWN!------------")
    return pd.concat(df_list).reset_index(drop=True)

### L1

In [20]:
#Oemt = pd.read_pickle("Order_ets_BU.pkl")
#Olmt = pd.read_pickle("Order_lgb_BU.pkl")
lead_time = 1
method = "ols"
MinT = pd.read_pickle(f"721OrderMinT_{method}_L{lead_time}.pkl") #OrderMinT.pkl#############################################################################
col = MinT.columns[-3:]
MinT[col] = MinT[col].clip(lower=0)
truth= pd.read_pickle(f"{path1}721future_28.pkl")['y'].iloc[ts_id.index]
Invt_df = []
names   = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu', 'ets_td', 'ets_mint']
#[ "lgb_base", "lgb_mint", "ets_base", "ets_mint"]
for i in names:
    print(f'_______________{i}')
    df = loop2(fcst = MinT[MinT['name'] == i].iloc[:,-3:], truth = truth,
               residual = [], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"721MinTOrder_{method}_L{lead_time}.pkl")

_______________lgb_base


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:53<00:00, 800.38it/s]


-----------DOWN!------------
_______________lgb_bu


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:53<00:00, 794.98it/s]


-----------DOWN!------------
_______________lgb_td


...

-----------DOWN!------------
_______________ets_td


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:52<00:00, 810.79it/s]


-----------DOWN!------------
_______________ets_mint


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:52<00:00, 815.68it/s]


-----------DOWN!------------


In [16]:
MinT#['name']#.unique()

,name,ot_90/MinTrace_method-ols,ot_95/MinTrace_method-ols,ot_99/MinTrace_method-ols
0,lgb_base,8.896450,10.925749,14.732374
1,lgb_base,3.130311,3.130311,3.130311
2,lgb_base,5.025634,5.025634,5.025634
3,lgb_base,8.046901,8.046901,8.046901
4,lgb_base,0.409922,0.409922,0.409922
...,...,...,...,...
9561659,ets_mint,0.042276,0.042276,0.042276
9561660,ets_mint,0.014360,0.014360,0.014360
9561661,ets_mint,0.025576,0.025576,0.025576
9561662,ets_mint,0.026471,0.026471,0.026471


In [15]:
a = 2
pd.concat(Invt_df).iloc[a*28:(a+1)*28,:]

,name,true_demand,forecasts,ot_90,ip_90,net_90,backlog_90,ch_90,cb_90,ot_95,...,net_95,backlog_95,ch_95,cb_95,ot_99,ip_99,net_99,backlog_99,ch_99,cb_99
56,lgb_base,7,0,3.906576,3.906576,3.906576,0.000000,3.906576,0.000000,5.673571,...,5.673571,0.000000,5.673571,0.000000,9.002294,9.002294,9.002294,0.0,9.002294,0.0
57,lgb_base,13,0,7.209276,-1.884148,-1.884148,1.884148,0.000000,16.957335,7.236432,...,-0.089997,0.089997,0.000000,1.709942,7.273239,3.275533,3.275533,0.0,3.275533,0.0
58,lgb_base,9,0,14.969354,4.085206,4.085206,0.000000,4.085206,0.000000,14.969354,...,5.879357,0.000000,5.879357,0.000000,14.969354,9.244887,9.244887,0.0,9.244887,0.0
59,lgb_base,1,0,9.512674,12.597880,12.597880,0.000000,12.597880,0.000000,9.512674,...,14.392031,0.000000,14.392031,0.000000,9.512674,17.757561,17.757561,0.0,17.757561,0.0
60,lgb_base,2,0,0.666132,11.264012,11.264012,0.000000,11.264012,0.000000,0.666132,...,13.058163,0.000000,13.058163,0.000000,0.666132,16.423693,16.423693,0.0,16.423693,0.0
61,lgb_base,4,0,2.473174,9.737186,9.737186,0.000000,9.737186,0.000000,2.473174,...,11.531338,0.000000,11.531338,0.000000,2.473174,14.896867,14.896867,0.0,14.896867,0.0
62,lgb_base,7,0,3.626984,6.364171,6.364171,0.000000,6.364171,0.000000,3.626984,...,8.158322,0.000000,8.158322,0.000000,3.626984,11.523852,11.523852,0.0,11.523852,0.0
63,lgb_base,9,0,6.679023,4.043194,4.043194,0.000000,4.043194,0.000000,6.679023,...,5.837345,0.000000,5.837345,0.000000,6.679023,9.202875,9.202875,0.0,9.202875,0.0
64,lgb_base,2,0,9.814723,11.857917,11.857917,0.000000,11.857917,0.000000,9.814723,...,13.652068,0.000000,13.652068,0.000000,9.814723,17.017598,17.017598,0.0,17.017598,0.0
65,lgb_base,3,0,0.268931,9.126847,9.126847,0.000000,9.126847,0.000000,0.268931,...,10.920999,0.000000,10.920999,0.000000,0.268931,14.286528,14.286528,0.0,14.286528,0.0


### L3

In [10]:
lead_time = 3
method = "ols"
MinT = pd.read_pickle(f"OrderMinT_L{lead_time}_{method}.pkl") #OrderMinT.pkl#############################################################################
col = MinT.columns[-3:]
MinT[col] = MinT[col].clip(lower=0)
truth= pd.read_pickle(f"{path1}future_28.pkl")['y'].iloc[ts_id.index]
Invt_df = []
names   = [ "lgb_base", "lgb_mint", "ets_base", "ets_mint"]
for i in names:
    df = loop2(fcst = MinT[MinT['name'] == i].iloc[:,-3:], truth = truth,
               residual = [], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"MinTOrder_L{lead_time}_{method}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:56<00:00, 749.45it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:56<00:00, 750.59it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:57<00:00, 744.88it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:56<00:00, 750.61it/s]


-----------DOWN!------------


### L7

In [11]:
lead_time = 7
method = "ols"
MinT = pd.read_pickle(f"OrderMinT_L{lead_time}_{method}.pkl") #OrderMinT.pkl#############################################################################
col = MinT.columns[-3:]
MinT[col] = MinT[col].clip(lower=0)
truth= pd.read_pickle(f"{path1}future_28.pkl")['y'].iloc[ts_id.index]
Invt_df = []
names   = [ "lgb_base", "lgb_mint", "ets_base", "ets_mint"]
for i in names:
    df = loop2(fcst = MinT[MinT['name'] == i].iloc[:,-3:], truth = truth,
               residual = [], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"MinTOrder_L{lead_time}_{method}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:03<00:00, 676.88it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:02<00:00, 680.67it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:03<00:00, 677.00it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:02<00:00, 684.31it/s]


-----------DOWN!------------
